In [4]:
import sqlite3
import sys
import os

pwd = os.getcwd()
items_database = os.path.join(pwd, 'mi_items.db')

conn = sqlite3.connect(items_database)

c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_from_mi_store INTEGER NOT NULL UNIQUE,
    name TEXT NOT NULL,
    number_of_comments INTEGER NOT NULL DEFAULT 0
)
''')

# 创建另一个表，comments
c.execute('''
CREATE TABLE IF NOT EXISTS comments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    item_id INTEGER NOT NULL,
    date TEXT NOT NULL,
    name TEXT NOT NULL,
    comment_count INTEGER NOT NULL DEFAULT 0,
    FOREIGN KEY (item_id) REFERENCES items (id)
)
''')

# 创建另一个表，已经下架的商品 off_shelf_items
c.execute('''
CREATE TABLE IF NOT EXISTS off_shelf_items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_from_mi_store INTEGER NOT NULL UNIQUE,
    name TEXT NOT NULL,
    number_of_comments INTEGER NOT NULL DEFAULT 0
)
''')

# 提交事务
conn.commit()

# 关闭连接
conn.close()



In [62]:
import sqlite3
def get_table_columns(data_base:str='mi_items.db'):
    # 读取数据库中的每个表，大小，列名
    conn = sqlite3.connect(data_base)
    cursor = conn.cursor()
    # 打印所有表名
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table in tables:
        print(table)
        # 打印表的列名
        cursor.execute("PRAGMA table_info(%s)" % table)
        columns = cursor.fetchall()
        for column in columns:
            print(column)
        print()
    conn.close()



def query_all_items(data_base:str='mi_items.db'):
    conn = sqlite3.connect(data_base)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM items')
    items = cursor.fetchall()  # 获取所有结果
    conn.close()
    return items

def query_all_comments():
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    c.execute('''
    SELECT * FROM comments
    ''')
    comments = c.fetchall()
    conn.close()
    return comments

def query_comments_by_name(name):
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    
    # 从comments表中查询数据，条件是name字段等于传入的name参数
    c.execute('''
    SELECT * FROM comments WHERE name = ?
    ''', (name,))
    comments = c.fetchall()
    conn.close()
    return comments

for comment in query_all_comments():
    print(comment)

# 打印所有项目
for item in query_all_items("mi_items.db"):
    print(item)

get_table_columns('mi_items.db')

(219, 285, '2024-3-1', 'Redmi Note 13 5G', 1432655)
(220, 359, '2024-3-1', 'Xiaomi 14', 1581318)
(221, 360, '2024-3-1', 'Redmi K70', 688004)
(222, 285, '2024-3-2', 'Redmi Note 13 5G', 1447934)
(223, 359, '2024-3-2', 'Xiaomi 14', 1592735)
(224, 360, '2024-3-2', 'Redmi K70', 698298)
(225, 285, '2024-3-3', 'Redmi Note 13 5G', 1463465)
(226, 359, '2024-3-3', 'Xiaomi 14', 1604573)
(227, 360, '2024-3-3', 'Redmi K70', 708921)
(228, 285, '2024-3-4', 'Redmi Note 13 5G', 1478263)
(229, 359, '2024-3-4', 'Xiaomi 14', 1617082)
(230, 360, '2024-3-4', 'Redmi K70', 720400)
(231, 285, '2024-3-5', 'Redmi Note 13 5G', 1490602)
(232, 359, '2024-3-5', 'Xiaomi 14', 1626644)
(233, 360, '2024-3-5', 'Redmi K70', 729579)
(234, 285, '2024-3-6', 'Redmi Note 13 5G', 1502002)
(235, 359, '2024-3-6', 'Xiaomi 14', 1636015)
(236, 360, '2024-3-6', 'Redmi K70', 739940)
(237, 285, '2024-3-7', 'Redmi Note 13 5G', 1512597)
(238, 359, '2024-3-7', 'Xiaomi 14', 1644881)
(239, 360, '2024-3-7', 'Redmi K70', 746580)
(240, 285, '2

In [43]:
def insert_item(id_mi, name, number_of_comments=0):
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    try:
        c.execute('''
        INSERT INTO items (id_from_mi_store, name, number_of_comments)
        VALUES (?, ?, ?)
        ''', (id_mi, name, number_of_comments))
        conn.commit()
    except sqlite3.IntegrityError as e:
        print("id_mi already exists, update it")
        c.execute('''
        UPDATE items
        SET name = ?, number_of_comments = ?
        WHERE id_from_mi_store = ?
        ''', (name, number_of_comments, id_mi))
        conn.commit()
    conn.close()

def insert_comments(id_mi,date,name,comment_count):
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    # get item_id from items table
    c.execute('''
    SELECT id FROM items WHERE id_from_mi_store = ?
    ''', (id_mi,))
    item_id = c.fetchall()
    if len(item_id) == 0:
        print("no such item")
        return

    item_id = item_id[0][0]
    # insert into comments table
    c.execute('''
    INSERT INTO comments (item_id, date, name, comment_count)
    VALUES (?, ?, ?, ?)
    ''', (item_id, date, name, comment_count))
    conn.commit()
   
    # update number_of_comments in items table to comments count
    c.execute('''
    UPDATE items
    SET number_of_comments = ?
    WHERE id = ?
    ''', (comment_count, item_id))
    conn.commit()
    conn.close()


def delete_item(c, id_mi):
    # 使用传入的 cursor 执行删除操作
    c.execute('''
    DELETE FROM items WHERE id_from_mi_store = ?
    ''', (id_mi,))

def insert_off_shelf_item(id_mi, name):
    # 使用 with 语句确保数据库连接正确关闭
    with sqlite3.connect('mi_items.db') as conn:
        c = conn.cursor()

        # 获取 items 表中的 number_of_comments
        c.execute('''
        SELECT number_of_comments FROM items WHERE id_from_mi_store = ?
        ''', (id_mi,))
        number_of_comments = c.fetchall()

        if len(number_of_comments) == 0:
            print("no such item")
            return

        number_of_comments = number_of_comments[0][0]

        # 插入数据到 off_shelf_items 表
        c.execute('''
        INSERT INTO off_shelf_items (id_from_mi_store, name, number_of_comments)
        VALUES (?, ?, ?)
        ''', (id_mi, name, number_of_comments))
        
        # 调用 delete_item 函数，并传入 cursor 和 id_mi
        delete_item(c, id_mi)

        # 提交事务
        conn.commit()

def clear_datebase():
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    c.execute('''
    DELETE FROM items
    ''')
    c.execute('''
    DELETE FROM comments
    ''')
    c.execute('''
    DELETE FROM off_shelf_items
    ''')
    conn.commit()
    conn.close()

def clear_comments():
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    c.execute('''
    DELETE FROM comments
    ''')
    conn.commit()
    conn.close()

def search_mi_id_by_name(name):
    conn = sqlite3.connect('mi_items.db')
    c = conn.cursor()
    c.execute('''
    SELECT id_from_mi_store FROM items WHERE name = ?
    ''', (name,))
    id_mi = c.fetchall()
    conn.close()
    return id_mi[0][0] if len(id_mi) > 0 else None

# 添加mi_id 和name到数据库

In [38]:
# 从之前的数据集读取数据，添加到数据库中


conn = sqlite3.connect('../mi_items.db')
c = conn.cursor()
c.execute('''
SELECT * FROM items
''')
items = c.fetchall()
conn.close()

for item in items:
    insert_item(item[1], item[2], item[3])

In [40]:
import numpy as np
import pandas
# 读取history_data/csv文件
def read_csv(file_path):
    if not os.path.exists(file_path):
        print("file not found: %s" % file_path)
        sys.exit(1)
    return pandas.read_csv(file_path)
history_data = read_csv('../history_data/csv')

print(history_data)
    

     日期  Redmi Note 13 5G  Note13系列增量  Xiaomi 14   14系列增量  Redmi K70  K70系列新增  \
0   3/1         1432655.0         NaN    1581318      NaN   688004.0      NaN   
1   3/2         1447934.0     15279.0    1592735  11417.0   698298.0  10294.0   
2   3/3         1463465.0     15531.0    1604573  11838.0   708921.0  10623.0   
3   3/4         1478263.0     14798.0    1617082  12509.0   720400.0  11479.0   
4   3/5         1490602.0     12339.0    1626644   9562.0   729579.0   9179.0   
..  ...               ...         ...        ...      ...        ...      ...   
63  5/3         1901304.0      4828.0    2128678   8627.0  1007366.0   3121.0   
64  5/4         1907835.0      6531.0    2138493   9815.0  1011518.0   4152.0   
65  5/5         1912294.0      4459.0    2145041   6548.0  1014432.0   2914.0   
66  5/6         1919179.0      6885.0    2155626  10585.0  1018890.0   4458.0   
67  5/7         1925581.0      6402.0    2166190  10564.0  1022968.0   4078.0   

        14u总   14u新增  Redmi

In [48]:
mi_ids = []
mi_ids.append(search_mi_id_by_name('Redmi Note 13 5G'))
mi_ids.append(search_mi_id_by_name('Xiaomi 14'))
mi_ids.append(search_mi_id_by_name('Redmi K70'))
mi_ids.append(search_mi_id_by_name('Redmi Turbo 3'))

for i in range(len(history_data)):
    date = history_data.iloc[i, 0]
    # 转换date为标准格式 现在的格式是 5/12, 5是月份，转为 2024-05-12,也就是自动补充0
    date = '2024-' + date
    date = date.replace('/', '-')

    redmi_note_13_5g = history_data.iloc[i, 1]
    xiaomi_14 = history_data.iloc[i, 3]
    redmi_k70 = history_data.iloc[i, 5]
    redmi_turbo_3 = history_data.iloc[i, 9]
    
    if(not np.isnan(redmi_note_13_5g)):
        insert_comments(mi_ids[0], date, 'Redmi Note 13 5G', int(redmi_note_13_5g))
    if(not np.isnan(xiaomi_14)):
        insert_comments(mi_ids[1], date, 'Xiaomi 14', int(xiaomi_14))
    if(not np.isnan(redmi_k70)):
        insert_comments(mi_ids[2], date, 'Redmi K70', int(redmi_k70))
    if(not np.isnan(redmi_turbo_3)):
        insert_comments(mi_ids[3], date, 'Redmi Turbo 3', int(redmi_turbo_3))

# 从爬虫数据库中获取历史评论数据

In [69]:
def query_all_comments_old_data():
    conn = sqlite3.connect('../mi_items.db')
    c = conn.cursor()
    c.execute('''
    SELECT * FROM comments
    ''')
    comments = c.fetchall()
    conn.close()
    return comments

def get_name_from_id(item_id):
    conn = sqlite3.connect('../mi_items.db')
    c = conn.cursor()
    c.execute('''
    SELECT id_from_mi_store, name FROM items WHERE id = ?
    ''', (item_id,))
    item = c.fetchall()
    conn.close()
    return item[0][0],item[0][1] if len(item) > 0 else None

for count,comment in enumerate(query_all_comments_old_data()):
    if(count==0):
        continue
    print(comment)
    print(count )
    id_mi,name = get_name_from_id(comment[1])
    #def insert_comments(id_mi,date,name,comment_count):
    insert_comments(id_mi,comment[2],name,comment[3])

(2, 2, '2024-05-01', 838746)
1
(3, 3, '2024-05-01', 35682)
2
(4, 4, '2024-05-01', 19434)
3
(5, 5, '2024-05-01', 9932)
4
(6, 6, '2024-05-01', 16043)
5
(7, 7, '2024-05-01', 96783)
6
(8, 8, '2024-05-01', 112604)
7
(9, 9, '2024-05-01', 97)
8
(10, 10, '2024-05-01', 339980)
9
(11, 11, '2024-05-01', 61160)
10
(12, 12, '2024-05-01', 420)
11
(13, 13, '2024-05-01', 2292)
12
(14, 14, '2024-05-01', 24229)
13
(15, 15, '2024-05-01', 5033)
14
(16, 16, '2024-05-01', 75306)
15
(17, 17, '2024-05-01', 3729)
16
(18, 18, '2024-05-01', 3162)
17
(19, 19, '2024-05-01', 62173)
18
(20, 20, '2024-05-01', 62173)
19
(21, 21, '2024-05-01', 62173)
20
(22, 22, '2024-05-01', 62173)
21
(23, 23, '2024-05-01', 97855)
22
(24, 24, '2024-05-01', 97855)
23
(25, 25, '2024-05-01', 97855)
24
(26, 26, '2024-05-01', 97855)
25
(27, 27, '2024-05-01', 1087161)
26
(28, 28, '2024-05-01', 874)
27
(29, 29, '2024-05-01', 7604)
28
(30, 30, '2024-05-01', 1045)
29
(31, 31, '2024-05-01', 3371)
30
(32, 32, '2024-05-01', 3452)
31
(33, 33, '202

# TEST

In [7]:
# test
insert_item(1, 'mi 11', 100)
insert_item(2, 'mi 10', 200)
insert_comments(1, '2021-01-01', 'mi 11', 100)
insert_comments(2, '2021-01-01', 'mi 10', 200)

for item in query_all_items("mi_items.db"):
    print(item)

for comment in query_all_comments():
    print(comment)

(1, 1, 'mi 11', 100)
(2, 2, 'mi 10', 200)
(1, 1, '2021-01-01', 'mi 11', 100)
(2, 2, '2021-01-01', 'mi 10', 200)


In [8]:

insert_off_shelf_item(1, 'mi 11')

In [58]:
for item in query_all_items("mi_items.db"):
    print(item)

for comment in query_all_comments():
    print(comment)

(183, 19011, '米家电饭煲C1Pro 4L', 23424)
(184, 19013, 'Xiaomi Pad 6 Max 14', 854515)
(185, 19014, '小米焦点触控笔', 38740)
(186, 19015, 'Xiaomi Pad 6 Max 智能触控键盘', 19689)
(187, 19016, 'Xiaomi Pad 6 Max 磁吸双面保护壳', 10237)
(188, 19017, 'Xiaomi Pad 6 Max 抗反射钢化保护膜', 16667)
(189, 19019, 'Xiaomi MIX Fold 3', 99779)
(190, 19021, '智能家庭屏Pro 8+中枢网关套装', 115039)
(191, 19023, 'Redmi 12R', 97)
(192, 19026, 'Redmi K60 至尊版', 340086)
(193, 19027, '小米无线鼠标3 彩色版', 64946)
(194, 19030, '米家空气净化器4 Pro H滤芯', 433)
(195, 19031, '巨省电 米家空调 立式5匹新3级能效', 2538)
(196, 19054, 'Redmi 12 5G', 24731)
(197, 19055, 'Xiaomi MIX Fold 3 伸缩腕带素皮保护壳', 5263)
(198, 19056, '米家理发器2', 78333)
(199, 19057, '巨省电 米家空调 2匹新3级能效', 3899)
(200, 19058, '日常元素超超柔乳霜纸面巾', 3380)
(201, 19063, 'Pad 6 Max 8G+256G 智能键盘套装', 63270)
(202, 19064, 'Pad 6 Max 12G+256G 智能键盘套装', 63270)
(203, 19065, 'Pad 6 Max 12G+512G 智能键盘套装', 63270)
(204, 19070, 'Pad 6 Max 16G+1TB 智能键盘套装', 63270)
(205, 19071, 'Pad 6 Max 8G+256G 触控笔+键盘套装', 102010)
(206, 19072, 'Pad 6 Max 12G+256G 触控笔+键盘套装', 1

In [44]:
#clear_datebase()
clear_comments()

In [67]:
for comment in query_all_comments():
    print(comment)

(219, 285, '2024-3-1', 'Redmi Note 13 5G', 1432655)
(220, 359, '2024-3-1', 'Xiaomi 14', 1581318)
(221, 360, '2024-3-1', 'Redmi K70', 688004)
(222, 285, '2024-3-2', 'Redmi Note 13 5G', 1447934)
(223, 359, '2024-3-2', 'Xiaomi 14', 1592735)
(224, 360, '2024-3-2', 'Redmi K70', 698298)
(225, 285, '2024-3-3', 'Redmi Note 13 5G', 1463465)
(226, 359, '2024-3-3', 'Xiaomi 14', 1604573)
(227, 360, '2024-3-3', 'Redmi K70', 708921)
(228, 285, '2024-3-4', 'Redmi Note 13 5G', 1478263)
(229, 359, '2024-3-4', 'Xiaomi 14', 1617082)
(230, 360, '2024-3-4', 'Redmi K70', 720400)
(231, 285, '2024-3-5', 'Redmi Note 13 5G', 1490602)
(232, 359, '2024-3-5', 'Xiaomi 14', 1626644)
(233, 360, '2024-3-5', 'Redmi K70', 729579)
(234, 285, '2024-3-6', 'Redmi Note 13 5G', 1502002)
(235, 359, '2024-3-6', 'Xiaomi 14', 1636015)
(236, 360, '2024-3-6', 'Redmi K70', 739940)
(237, 285, '2024-3-7', 'Redmi Note 13 5G', 1512597)
(238, 359, '2024-3-7', 'Xiaomi 14', 1644881)
(239, 360, '2024-3-7', 'Redmi K70', 746580)
(240, 285, '2

In [71]:
data = query_comments_by_name('Xiaomi 14')
for comment in data:
    print(comment[2], comment[4])

2024-3-1 1581318
2024-3-2 1592735
2024-3-3 1604573
2024-3-4 1617082
2024-3-5 1626644
2024-3-6 1636015
2024-3-7 1644881
2024-3-8 1652040
2024-3-9 1659463
2024-3-10 1668483
2024-3-11 1679130
2024-3-12 1689060
2024-3-13 1696772
2024-3-14 1732999
2024-3-15 1749439
2024-3-16 1760043
2024-3-17 1770548
2024-3-18 1781937
2024-3-19 1792865
2024-3-20 1800064
2024-3-21 1807374
2024-3-22 1814630
2024-3-23 1821833
2024-3-24 1830320
2024-3-25 1839222
2024-3-26 1847764
2024-3-27 1853874
2024-3-28 1859764
2024-3-29 1865437
2024-3-30 1871111
2024-3-31 1877788
2024-4-1 1885970
2024-4-2 1893831
2024-4-3 1899468
2024-4-4 1905519
2024-4-5 1911751
2024-4-6 1917378
2024-4-7 1923299
2024-4-8 1930486
2024-4-9 1937130
2024-4-10 1942163
2024-4-11 1947352
2024-4-12 1951860
2024-4-13 1956962
2024-4-14 1963226
2024-4-15 1970611
2024-4-16 1980241
2024-4-17 1984450
2024-4-18 1988883
2024-4-19 1998987
2024-4-20 2011765
2024-4-21 2022933
2024-4-22 2034959
2024-4-23 2043575
2024-4-24 2051950
2024-4-25 2059185
2024-4-26 